# ORGANIZANDO OS DADOS DA MALHA VIÁRIA DE VIÇOSA - COM O ANEL

### IMPORTANDO O PACOTE psycopg2 QUE CONECTA O PYTHON COM O POSTGRE-SQL

In [1]:
import psycopg2 as pg
#import pandas as pd

### CONECTANDO AO BANCO DE DADOS

In [2]:
con = pg.connect(host='localhost', 
                database='dissertacao',
                user='postgres', 
                password='admin')

cur = con.cursor() #CRIANDO UMA INSTÂNCIA PARA EXECUTAR COMANDOS EM SQL

# OBS: O servidor hospedado na máquina local será conectado no banco de dados nomeado DISSERTAÇÃO, que possui usuário postgres e senha admin.

### VENDO QUANTOS GRAFOS A MALHA POSSUI

In [3]:
tabela_grafos = 'vias_grafos_anel' #TABELA COM OS GRAFOS
sql = f'select max(id) from {tabela_grafos}' #COMANDO EM SQL A SER EXECUTADO
cur.execute(sql) #EXECUTANDO O COMANDO CRIADO
dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
id_max = dados_consultados[0][0] #ID MÁXIMO DA MALHA

### ATRIBUINDO NOME DAS VIAS PARA O BANCO DE DADOS DOS GRAFOS

In [4]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos_anel = 'vias_grafos_anel' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA COM O ANEL
tabela_grafos = 'vias_grafos' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos_a = 'vga' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA COM O ANEL
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA

#CONSULTANDO OS DADOS DE CADA GRAFO DO MALHA COM ANEL VIARIO:

for id_i in range(1, id_max + 1):

    sql = f"SELECT {abr_grafos}.id FROM {tabela_grafos} {abr_grafos}, {tabela_grafos_anel} {abr_grafos_a} WHERE ST_Contains({abr_grafos}.geom, {abr_grafos_a}.geom) and {abr_grafos_a}.id = {id_i}" #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ID DA TABELA DOS DOS GRAFOS (QUE JÁ ESTÁ COMPLETA).

    cur.execute(sql) #EXECUTANDO O COMANDO

    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS

    #INSERINDO AS INFORMAÇÕES CONSULTADAS:

    if dados_consultados == []: #CONFERINDO SE A CONSULTA É VAZIA. CASO SEJA, PULA A ITERAÇÃO.
        print(f'ID grafo anel: {id_i} -> Passa!\n')
        continue        

    else:
        sql = f"UPDATE {tabela_grafos_anel} SET id_grafo_semanel = '{dados_consultados[0][0]}' WHERE id = {id_i}" #COMANDO EM SQL A SER EXECUTADO. SERÁ ADICIONADO OS ATRIBUTOS CONSULTADOS NA MALHA COM ANEL VIARIO.

        cur.execute(sql) #EXECUTANDO O COMANDO

        con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

        print(f'ID grafo anel: {id_i} -> ID grafo sem anel: {dados_consultados[0][0]}\n')


ID grafo anel: 1 -> Passa!

ID grafo anel: 2 -> Passa!

ID grafo anel: 3 -> Passa!

ID grafo anel: 4 -> ID grafo sem anel: 3

ID grafo anel: 5 -> ID grafo sem anel: 1

ID grafo anel: 6 -> ID grafo sem anel: 2

ID grafo anel: 7 -> ID grafo sem anel: 4

ID grafo anel: 8 -> ID grafo sem anel: 5

ID grafo anel: 9 -> ID grafo sem anel: 6

ID grafo anel: 10 -> ID grafo sem anel: 7

ID grafo anel: 11 -> ID grafo sem anel: 8

ID grafo anel: 12 -> ID grafo sem anel: 9

ID grafo anel: 13 -> Passa!

ID grafo anel: 14 -> ID grafo sem anel: 11

ID grafo anel: 15 -> ID grafo sem anel: 12

ID grafo anel: 16 -> ID grafo sem anel: 13

ID grafo anel: 17 -> ID grafo sem anel: 14

ID grafo anel: 18 -> ID grafo sem anel: 15

ID grafo anel: 19 -> ID grafo sem anel: 16

ID grafo anel: 20 -> ID grafo sem anel: 17

ID grafo anel: 21 -> ID grafo sem anel: 18

ID grafo anel: 22 -> ID grafo sem anel: 19

ID grafo anel: 23 -> ID grafo sem anel: 20

ID grafo anel: 24 -> ID grafo sem anel: 21

ID grafo anel: 25 -> I

### ATRIBUINDO NOME DAS VIAS PARA O BANCO DE DADOS DOS GRAFOS

In [5]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos_anel' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'nome_rua' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_nomeruas' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vnr' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'nome_rua' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO O NOME DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX
    
    sql = f'select {abr_dados}.{atributo_dados} from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
    
    for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
        if dado[0] not in lista_dados:
            lista_dados.append(dado[0])
    
    lista_dados_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS SERÃO TRANSFORMADA EM UMA STRING DA SEGUINTE FORMA: ATRIBUTO_1; ATRIBUTO_2; ATRIBUTO_3; ... ; ATRIBUTO_n
    
    for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
        if dado != lista_dados[len(lista_dados) - 1]:
            lista_dados_str = lista_dados_str + dado + '; '
        else:
            lista_dados_str = lista_dados_str + dado
    
    #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:
    
    sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{lista_dados_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
    cur.execute(sql) #EXECUTANDO O COMANDO
    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO
    
    print(f'ID: {id_i} {atributo_dados_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.

ID: 1 nome_rua OK!
ID: 2 nome_rua OK!
ID: 3 nome_rua OK!
ID: 4 nome_rua OK!
ID: 5 nome_rua OK!
ID: 6 nome_rua OK!
ID: 7 nome_rua OK!
ID: 8 nome_rua OK!
ID: 9 nome_rua OK!
ID: 10 nome_rua OK!
ID: 11 nome_rua OK!
ID: 12 nome_rua OK!
ID: 13 nome_rua OK!
ID: 14 nome_rua OK!
ID: 15 nome_rua OK!
ID: 16 nome_rua OK!
ID: 17 nome_rua OK!
ID: 18 nome_rua OK!
ID: 19 nome_rua OK!
ID: 20 nome_rua OK!
ID: 21 nome_rua OK!
ID: 22 nome_rua OK!
ID: 23 nome_rua OK!
ID: 24 nome_rua OK!
ID: 25 nome_rua OK!
ID: 26 nome_rua OK!
ID: 27 nome_rua OK!
ID: 28 nome_rua OK!
ID: 29 nome_rua OK!
ID: 30 nome_rua OK!
ID: 31 nome_rua OK!
ID: 32 nome_rua OK!
ID: 33 nome_rua OK!
ID: 34 nome_rua OK!
ID: 35 nome_rua OK!
ID: 36 nome_rua OK!
ID: 37 nome_rua OK!
ID: 38 nome_rua OK!
ID: 39 nome_rua OK!
ID: 40 nome_rua OK!
ID: 41 nome_rua OK!
ID: 42 nome_rua OK!
ID: 43 nome_rua OK!
ID: 44 nome_rua OK!
ID: 45 nome_rua OK!
ID: 46 nome_rua OK!
ID: 47 nome_rua OK!
ID: 48 nome_rua OK!
ID: 49 nome_rua OK!
ID: 50 nome_rua OK!
ID: 51 no

### ATRIBUINDO O TIPO DA VIA DE ACORDO COM O PLANO DE MOBILIDADE PARA O BANCO DE DADOS DOS GRAFOS

In [18]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos_anel' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'tipo_pm' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_tipopm' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vpm' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'tipo_pm' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO O TIPO DA VIA DE ACORDO COM O PLANO DE MOBILIDADE DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX
    
    sql = f'select {abr_dados}.{atributo_dados} from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
    
    for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
        if dado[0] not in lista_dados:
            lista_dados.append(dado[0])
    
    lista_dados_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS SERÃO TRANSFORMADA EM UMA STRING DA SEGUINTE FORMA: ATRIBUTO_1; ATRIBUTO_2; ATRIBUTO_3; ... ; ATRIBUTO_n
    
    for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
        if dado != lista_dados[len(lista_dados) - 1]:
            lista_dados_str = lista_dados_str + dado + '; '
        else:
            lista_dados_str = lista_dados_str + dado
    
    #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:
    
    sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{lista_dados_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
    cur.execute(sql) #EXECUTANDO O COMANDO
    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO
    
    print(f'ID: {id_i} {atributo_dados_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.

ID: 1 tipo_pm OK!
ID: 2 tipo_pm OK!
ID: 3 tipo_pm OK!
ID: 4 tipo_pm OK!
ID: 5 tipo_pm OK!
ID: 6 tipo_pm OK!
ID: 7 tipo_pm OK!
ID: 8 tipo_pm OK!
ID: 9 tipo_pm OK!
ID: 10 tipo_pm OK!
ID: 11 tipo_pm OK!
ID: 12 tipo_pm OK!
ID: 13 tipo_pm OK!
ID: 14 tipo_pm OK!
ID: 15 tipo_pm OK!
ID: 16 tipo_pm OK!
ID: 17 tipo_pm OK!
ID: 18 tipo_pm OK!
ID: 19 tipo_pm OK!
ID: 20 tipo_pm OK!
ID: 21 tipo_pm OK!
ID: 22 tipo_pm OK!
ID: 23 tipo_pm OK!
ID: 24 tipo_pm OK!
ID: 25 tipo_pm OK!
ID: 26 tipo_pm OK!
ID: 27 tipo_pm OK!
ID: 28 tipo_pm OK!
ID: 29 tipo_pm OK!
ID: 30 tipo_pm OK!
ID: 31 tipo_pm OK!
ID: 32 tipo_pm OK!
ID: 33 tipo_pm OK!
ID: 34 tipo_pm OK!
ID: 35 tipo_pm OK!
ID: 36 tipo_pm OK!
ID: 37 tipo_pm OK!
ID: 38 tipo_pm OK!
ID: 39 tipo_pm OK!
ID: 40 tipo_pm OK!
ID: 41 tipo_pm OK!
ID: 42 tipo_pm OK!
ID: 43 tipo_pm OK!
ID: 44 tipo_pm OK!
ID: 45 tipo_pm OK!
ID: 46 tipo_pm OK!
ID: 47 tipo_pm OK!
ID: 48 tipo_pm OK!
ID: 49 tipo_pm OK!
ID: 50 tipo_pm OK!
ID: 51 tipo_pm OK!
ID: 52 tipo_pm OK!
ID: 53 tipo_pm OK!
ID

### ATRIBUINDO O TIPO DA VIA DE ACORDO COM O PLANO DE MOBILIDADE PARA O BANCO DE DADOS DOS GRAFOS

In [19]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos_anel' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'tipo_pm' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_tipopm' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vpm' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'tipo_pm' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO O TIPO DA VIA DE ACORDO COM O PLANO DE MOBILIDADE DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX

    sql = f'select {abr_dados}.{atributo_dados} from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
    
    for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
        if dado[0] not in lista_dados:
            lista_dados.append(dado[0])
    
    lista_dados_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS SERÃO TRANSFORMADA EM UMA STRING DA SEGUINTE FORMA: ATRIBUTO_1; ATRIBUTO_2; ATRIBUTO_3; ... ; ATRIBUTO_n
    
    for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
        if dado != lista_dados[len(lista_dados) - 1]:
            lista_dados_str = lista_dados_str + dado + '; '
        else:
            lista_dados_str = lista_dados_str + dado
    
    #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:
    
    sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{lista_dados_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
    cur.execute(sql) #EXECUTANDO O COMANDO
    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO
    
    print(f'ID: {id_i} {atributo_dados_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.

ID: 1 tipo_pm OK!
ID: 2 tipo_pm OK!
ID: 3 tipo_pm OK!
ID: 4 tipo_pm OK!
ID: 5 tipo_pm OK!
ID: 6 tipo_pm OK!
ID: 7 tipo_pm OK!
ID: 8 tipo_pm OK!
ID: 9 tipo_pm OK!
ID: 10 tipo_pm OK!
ID: 11 tipo_pm OK!
ID: 12 tipo_pm OK!
ID: 13 tipo_pm OK!
ID: 14 tipo_pm OK!
ID: 15 tipo_pm OK!
ID: 16 tipo_pm OK!
ID: 17 tipo_pm OK!
ID: 18 tipo_pm OK!
ID: 19 tipo_pm OK!
ID: 20 tipo_pm OK!
ID: 21 tipo_pm OK!
ID: 22 tipo_pm OK!
ID: 23 tipo_pm OK!
ID: 24 tipo_pm OK!
ID: 25 tipo_pm OK!
ID: 26 tipo_pm OK!
ID: 27 tipo_pm OK!
ID: 28 tipo_pm OK!
ID: 29 tipo_pm OK!
ID: 30 tipo_pm OK!
ID: 31 tipo_pm OK!
ID: 32 tipo_pm OK!
ID: 33 tipo_pm OK!
ID: 34 tipo_pm OK!
ID: 35 tipo_pm OK!
ID: 36 tipo_pm OK!
ID: 37 tipo_pm OK!
ID: 38 tipo_pm OK!
ID: 39 tipo_pm OK!
ID: 40 tipo_pm OK!
ID: 41 tipo_pm OK!
ID: 42 tipo_pm OK!
ID: 43 tipo_pm OK!
ID: 44 tipo_pm OK!
ID: 45 tipo_pm OK!
ID: 46 tipo_pm OK!
ID: 47 tipo_pm OK!
ID: 48 tipo_pm OK!
ID: 49 tipo_pm OK!
ID: 50 tipo_pm OK!
ID: 51 tipo_pm OK!
ID: 52 tipo_pm OK!
ID: 53 tipo_pm OK!
ID

### ATRIBUINDO O SENTIDO DA VIA PARA O BANCO DE DADOS DOS GRAFOS

In [20]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos_anel' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'oneway' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_oneway' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vow' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'oneway' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO SENTIDO DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX
    
    sql = f'select {abr_dados}.{atributo_dados} from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
    
    for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
        if dado[0] not in lista_dados:
            lista_dados.append(dado[0])
    
    lista_dados_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS SERÃO TRANSFORMADA EM UMA STRING DA SEGUINTE FORMA: ATRIBUTO_1; ATRIBUTO_2; ATRIBUTO_3; ... ; ATRIBUTO_n
    
    for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
        if dado != lista_dados[len(lista_dados) - 1]:
            lista_dados_str = lista_dados_str + dado + '; '
        else:
            lista_dados_str = lista_dados_str + dado
    
    #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:
    
    sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{lista_dados_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
    cur.execute(sql) #EXECUTANDO O COMANDO
    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO
    
    print(f'ID: {id_i} {atributo_dados_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.


ID: 1 oneway OK!
ID: 2 oneway OK!
ID: 3 oneway OK!
ID: 4 oneway OK!
ID: 5 oneway OK!
ID: 6 oneway OK!
ID: 7 oneway OK!
ID: 8 oneway OK!
ID: 9 oneway OK!
ID: 10 oneway OK!
ID: 11 oneway OK!
ID: 12 oneway OK!
ID: 13 oneway OK!
ID: 14 oneway OK!
ID: 15 oneway OK!
ID: 16 oneway OK!
ID: 17 oneway OK!
ID: 18 oneway OK!
ID: 19 oneway OK!
ID: 20 oneway OK!
ID: 21 oneway OK!
ID: 22 oneway OK!
ID: 23 oneway OK!
ID: 24 oneway OK!
ID: 25 oneway OK!
ID: 26 oneway OK!
ID: 27 oneway OK!
ID: 28 oneway OK!
ID: 29 oneway OK!
ID: 30 oneway OK!
ID: 31 oneway OK!
ID: 32 oneway OK!
ID: 33 oneway OK!
ID: 34 oneway OK!
ID: 35 oneway OK!
ID: 36 oneway OK!
ID: 37 oneway OK!
ID: 38 oneway OK!
ID: 39 oneway OK!
ID: 40 oneway OK!
ID: 41 oneway OK!
ID: 42 oneway OK!
ID: 43 oneway OK!
ID: 44 oneway OK!
ID: 45 oneway OK!
ID: 46 oneway OK!
ID: 47 oneway OK!
ID: 48 oneway OK!
ID: 49 oneway OK!
ID: 50 oneway OK!
ID: 51 oneway OK!
ID: 52 oneway OK!
ID: 53 oneway OK!
ID: 54 oneway OK!
ID: 55 oneway OK!
ID: 56 oneway OK!
I

### ATRIBUINDO A LARGURA DA VIA PARA O BANCO DE DADOS DOS GRAFOS

In [28]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos_anel' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'largura_me' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_largura' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vl' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'largura' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO A LARGURA DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX

    sql = f'select {abr_dados}.{atributo_dados}, st_length({abr_dados}.geom) from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    if dados_consultados == []: #CONFERINDO SE INFORMAÇÕES CONSULTADAS. SE NÃO EXISTIR, PULA A ITERAÇÃO.
        print(f'{id_i}: Pula!')
        continue
        
    else:
        numerador = 0 #CRIANDO UMA LISTA VAZIA, NO QUAL SERÁ SOMADO TODOS OS ATRIBUTOS PONDERADOS PELO TAMANHO DE SUA LINHA
        denominador = 0 #SOMA DOS PESOS

        for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NAS LISTAS CRIADAS
            numerador = numerador + dado[0]*dado[1]
            denominador = denominador + dado [1]

        largura_me_i = float(f'{numerador/denominador:.2f}') #CALCULANDO A LARGURA MEDIA DO GRAFO i

        #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:

        sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{largura_me_i}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
        cur.execute(sql) #EXECUTANDO O COMANDO
        con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

        print(f'ID: {id_i} {largura_me_i} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.


1: Pula!
ID: 2 6.19 OK!
ID: 3 6.44 OK!
ID: 4 6.05 OK!
ID: 5 4.31 OK!
ID: 6 5.51 OK!
7: Pula!
ID: 8 6.34 OK!
ID: 9 6.98 OK!
ID: 10 6.19 OK!
ID: 11 6.47 OK!
ID: 12 9.52 OK!
ID: 13 6.98 OK!
ID: 14 8.01 OK!
ID: 15 6.77 OK!
ID: 16 6.87 OK!
ID: 17 7.47 OK!
ID: 18 7.54 OK!
ID: 19 7.73 OK!
ID: 20 7.72 OK!
ID: 21 12.99 OK!
ID: 22 7.97 OK!
ID: 23 10.18 OK!
ID: 24 7.41 OK!
ID: 25 7.89 OK!
ID: 26 12.87 OK!
ID: 27 6.49 OK!
ID: 28 8.5 OK!
ID: 29 7.12 OK!
ID: 30 7.77 OK!
ID: 31 7.7 OK!
ID: 32 14.27 OK!
ID: 33 11.93 OK!
ID: 34 6.55 OK!
ID: 35 7.58 OK!
ID: 36 6.1 OK!
ID: 37 5.15 OK!
ID: 38 6.94 OK!
ID: 39 7.09 OK!
ID: 40 5.99 OK!
ID: 41 7.83 OK!
ID: 42 11.83 OK!
ID: 43 10.83 OK!
ID: 44 6.11 OK!
ID: 45 6.82 OK!
ID: 46 8.82 OK!
ID: 47 7.39 OK!
ID: 48 6.76 OK!
ID: 49 8.33 OK!
ID: 50 6.42 OK!
ID: 51 6.59 OK!
ID: 52 7.81 OK!
ID: 53 8.95 OK!
ID: 54 7.38 OK!
ID: 55 6.19 OK!
ID: 56 7.92 OK!
ID: 57 6.61 OK!
ID: 58 15.28 OK!
ID: 59 7.94 OK!
ID: 60 7.33 OK!
ID: 61 11.15 OK!
ID: 62 11.99 OK!
ID: 63 7.64 OK!
ID: 64

### ATRIBUINDO A INCLINAÇÃO MÉDIA PARA O BANCO DE DADOS DOS GRAFOS

In [19]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos_anel' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'decliv_me' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'mde_vicosa' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'mde' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO A DECLIVIDADE DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA
# OBS: O GRAFO DE ID = 1 SERÁ PULADO, UMA VEZ QUE O MDE NÃO POSSUI INFORMAÇÕES PARA ESSA LINHA
for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX
    
    if id_i == 7: #O GRAFO COM ID 7 NÃO POSSUI INFORMAÇÕES COMPLETAS DE MDE, PORTANTO SERÁ PULADO. PARA OS DEMAIS GRAFOS, SEGUE A ROTINA NORMALMENTE.
        
        print('Pulando o grafo de id 7.')
        continue
    
    else:
        #COMPRIMENTO DO GRAFO

        sql = f'SELECT ST_Length(geom) FROM {tabela_grafos} WHERE id = {id_i}' #COMANDO EM SQL PARA CONSULTAR O COMPRIMENTO DO GRAFO

        cur.execute(sql) #EXECUTANDO O COMANDO

        dados_consultados = cur.fetchall() #RETORNANDO OS DADOS

        comp_grafo = dados_consultados[0][0] #COMPRIMENTO DO GRAFO

        #SELECIONANDO AS ALTITUDES AO LONGO DO GRAFO DE ACORDO COM O COMPRIMENTO DO GRAFO:
    
        if comp_grafo <= 300: #CASO O COMPRIMENTO DO GRAFO SEJA <300m, O GRAFO SERÁ DIVIDIDO EM 4 PARTES:

            lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS

            for per in range(0, 101, 25):            

                sql = f"SELECT ST_Value({abr_dados}.rast, pts.geom) as elevacao FROM (SELECT ST_LineInterpolatePoint(ST_LineMerge(geom), {per/100}) as geom FROM {tabela_grafos} WHERE id = {id_i}) as pts, {tabela_dados} {abr_dados} WHERE ST_Intersects({abr_dados}.rast, pts.geom);" #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO AS COTAS DOS PONTOS AO LONGO DO GRAFO PARA O ID DO GRAFO EM ANÁLISE.

                cur.execute(sql) #EXECUTANDO O COMANDO

                dados_consultados = cur.fetchall() #RETORNANDO OS DADOS            

                lista_dados.append(dados_consultados[0][0])    

            #CALCULANDO A DECLIVIDADE MEDIA DO GRAFO
            decliv = 0
            for i in range(0, len(lista_dados)-1):
                decliv = decliv + (lista_dados[i+1] - lista_dados[i])/(comp_grafo/4)

            decliv = round(decliv/4*100, 2)

        elif comp_grafo <= 600: #SE O COMPRIMENTO DO GRAFO FOR <600m ENTÃO O GRAFO SERÁ DIVIDIDO EM 10 PARTES:

            lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS

            for per in range(0, 101, 10):            

                sql = f"SELECT ST_Value({abr_dados}.rast, pts.geom) as elevacao FROM (SELECT ST_LineInterpolatePoint(ST_LineMerge(geom), {per/100}) as geom FROM {tabela_grafos} WHERE id = {id_i}) as pts, {tabela_dados} {abr_dados} WHERE ST_Intersects({abr_dados}.rast, pts.geom);" #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO AS COTAS DOS PONTOS AO LONGO DO GRAFO PARA O ID DO GRAFO EM ANÁLISE.

                cur.execute(sql) #EXECUTANDO O COMANDO

                dados_consultados = cur.fetchall() #RETORNANDO OS DADOS            

                lista_dados.append(dados_consultados[0][0])    

            #CALCULANDO A DECLIVIDADE MEDIA DO GRAFO
            decliv = 0
            for i in range(0, len(lista_dados)-1):
                decliv = decliv + (lista_dados[i+1] - lista_dados[i])/(comp_grafo/10)

            decliv = round(decliv/10*100, 2)

        else:        

            lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS

            for per in range(0, 101, 5):            

                sql = f"SELECT ST_Value({abr_dados}.rast, pts.geom) as elevacao FROM (SELECT ST_LineInterpolatePoint(ST_LineMerge(geom), {per/100}) as geom FROM {tabela_grafos} WHERE id = {id_i}) as pts, {tabela_dados} {abr_dados} WHERE ST_Intersects({abr_dados}.rast, pts.geom);" #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO AS COTAS DOS PONTOS AO LONGO DO GRAFO PARA O ID DO GRAFO EM ANÁLISE.

                cur.execute(sql) #EXECUTANDO O COMANDO

                dados_consultados = cur.fetchall() #RETORNANDO OS DADOS            

                lista_dados.append(dados_consultados[0][0])    

            #CALCULANDO A DECLIVIDADE MEDIA DO GRAFO
            decliv = 0
            for i in range(0, len(lista_dados)-1):
                decliv = decliv + (lista_dados[i+1] - lista_dados[i])/(comp_grafo/20)

            decliv = round(decliv/20*100, 2)

        #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:

        sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{decliv}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
        cur.execute(sql) #EXECUTANDO O COMANDO
        con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

        print(f'ID: {id_i} {decliv} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.
        
cur.close() #ENCERRANDO A INSTÂNCIA CRIADA PARA A EXECUÇÃO DO COMANDO
con.close() #ENCERRANDO A CONEXÃO COM O BANCO DE DADOS


ID: 1 -0.05 OK!
ID: 2 -15.82 OK!
ID: 3 -2.17 OK!
ID: 4 0.8 OK!
ID: 5 10.84 OK!
ID: 6 0.42 OK!
Pulando o grafo de id 7.
ID: 8 14.27 OK!
ID: 9 2.96 OK!
ID: 10 -6.9 OK!
ID: 11 -0.7 OK!
ID: 12 7.58 OK!
ID: 13 13.45 OK!
ID: 14 4.2 OK!
ID: 15 4.49 OK!
ID: 16 4.62 OK!
ID: 17 -1.19 OK!
ID: 18 1.48 OK!
ID: 19 -2.29 OK!
ID: 20 1.3 OK!
ID: 21 2.81 OK!
ID: 22 0.54 OK!
ID: 23 14.88 OK!
ID: 24 2.53 OK!
ID: 25 0.17 OK!
ID: 26 6.52 OK!
ID: 27 -10.46 OK!
ID: 28 -9.86 OK!
ID: 29 8.72 OK!
ID: 30 0.93 OK!
ID: 31 -2.9 OK!
ID: 32 4.19 OK!
ID: 33 6.13 OK!
ID: 34 9.38 OK!
ID: 35 -0.5 OK!
ID: 36 4.99 OK!
ID: 37 -4.1 OK!
ID: 38 -1.38 OK!
ID: 39 3.43 OK!
ID: 40 -3.03 OK!
ID: 41 -1.42 OK!
ID: 42 0.5 OK!
ID: 43 1.69 OK!
ID: 44 -0.62 OK!
ID: 45 -0.04 OK!
ID: 46 0.24 OK!
ID: 47 -1.36 OK!
ID: 48 1.7 OK!
ID: 49 0.75 OK!
ID: 50 -3.47 OK!
ID: 51 -14.03 OK!
ID: 52 5.1 OK!
ID: 53 2.74 OK!
ID: 54 0.51 OK!
ID: 55 9.1 OK!
ID: 56 0.8 OK!
ID: 57 18.32 OK!
ID: 58 -8.38 OK!
ID: 59 -3.15 OK!
ID: 60 7.43 OK!
ID: 61 0.39 OK!
ID: 62